In [1]:
import numpy as np

In [4]:
def log_likelihood(F, A):
    FRows, FCols = F.shape
    ARows, ACols = A.shape
    log_likelihood = 0
    for u in range(ARows):
        for v in range(ACols):
            if(u == v):
                continue
            Fs = 1
            for i in range(FCols):
                Fs *= exp(-1 * F[u, i] * F[v, i])

            a = A[u, v]
            log_likelihood += log(a + Fs - 2 * a * Fs)
    return log_likelihood

In [5]:
def by_rond_F_k(F, A, u, k):
    rond_F_k = 0
    FRows, FCols = F.shape
    ARows, ACols = A.shape
    for v in range(ACols):
        if(u == v):
            continue
        Fs = 1
        for i in range(FCols):
            Fs *= exp(-1 * F[u, i] * F[v, i])
        a = A[u, v]
        rond_F_k += (2*a - 1) * F[v, k] * Fs / (a + Fs - 2 * a * Fs)
    rond_F_k *= 2
    return rond_F_k

def gradient(F, A, u):
    gradient = []
    FRows, FCols = F.shape
    for k in range(FCols):
        gradient.push(by_rond_F_k(F, A, u, k))
    return gradient

In [6]:
def train(A, C, iterations = 200):
    # initialize an F
    N = A.shape[0]
    F = np.random.rand(N,C)

    for n in range(iterations):
        for person in range(N):
            grad = gradient(F, A, person)
            F[person] += 0.005*grad # updating F
            F[person] = np.maximum(0.001, F[person]) # F should be nonnegative
            ll = log_likelihood(F, A)
            print('At step %4i logliklihood is %5.4f'%(n,ll))

    return F